In [10]:
class LinkedQueue:
    class _Nodes:
        __slots__ = "_elements", "_next"

        def __init__(self, elements, next) -> None:
            self._elements = elements
            self._next = next

    def __init__(self) -> None:
        self._head = None
        self._tail = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def first(self):
        if self.is_empty():
            return "Empty"
        return self._head._elements

    def dequeue(self):
        if self.is_empty():
            return "Empty"
        answer = self._head._elements
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return answer

    def enqueue(self, e):
        newest = self._Nodes(e, None)

        if self.is_empty():
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1

    def __str__(self) -> str:
        elements = []
        current = self._head
        while current is not None:
            elements.append(str(current._elements))
            current = current._next

        return "Head->" + "->".join(elements) + "->Tail"

    def Last(self):
        if self.is_empty():
            return "empty"
        return self._tail._elements

# Trees


` with the Tree class being abstract, there is no reason to create a
 direct instance of it, nor would such an instance be useful. The class exists to serve
 as a base for inheritance, and users will create instances of concrete subclasses.`


In [11]:
class Tree:

    class Position:

        def element(self):

            return NotImplementedError("Must be implemented")

        def __eq__(self, other: object) -> bool:
            return NotImplementedError("must be implemented")

        def __ne__(self, other: object) -> bool:
            return not (self == other)

    def root(self):
        return NotImplementedError("Must be implemented")

    def parent(self, p):
        return NotImplementedError("Must be implemented")

    def num_children(self, p):
        return NotImplementedError("Must be implemented")

    def children(self, p):
        return NotImplementedError("Must be implemented")

    def __len__(self):
        return NotImplementedError("Must be implemented")

    def is_root(self, p):
        return self.root() == p

    def is_leaf(self, p):
        return self.num_children(p) == 0

    def is_empty(self):
        return len(self) == 0

    def depth(self, p):

        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _height2(self, p):

        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))

    def height(self, p=None):

        if p is None:
            p = self.root()
        return self._height2(p)

    def __iter__(self):
        for p in self.positions():
            yield p.element()

    def preorder(self):

        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p

    def _subtree_preorder(self, p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other

    def positions(self):
        return self.preorder()

    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p

    def _subtree_postorder(self, p):

        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p

    def breadthfirst(self):
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root())
            while not fringe.is_empty():
                p = fringe.dequeue()
                yield p
                for c in self.children(p):
                    fringe.enqueue(c)

In [12]:
class BinaryTree(Tree):

    def left(self, p):
        raise NotImplementedError("Must be implemented")

    def right(self, p):
        raise NotImplementedError("must be implemented")

    def sibling(self, p):

        parent = self.parent(p)  # here we get the node from which this one originates

        if parent is None:  # root node that is
            return None
        else:  # if any node in tree or subtree

            if p == self.left(parent):
                return self.right(
                    parent
                )  # after we got the parent [ in binary tree there are only two nodes so P can be either left or right and whichever it matches we return the opposite one ]
            else:
                return self.left(parent)

    def children(
        self, p
    ):  # here we take p as positon of a parent and provide simply the nodes connenceted to it.... must not be that hard to understand......teeeeeeeeeeeee

        if self.left(p) is not None:
            yield self.left(p)

        if self.right(p) is not None:
            yield self.right(p)

    def inorder(self):
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p

    def _subtree_inorder(self, p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other

    def positions(self):
        return self.inorder()

In [13]:
class LinkedBinaryTree(BinaryTree):

    class _Node:
        __slots__ = "_element", "_parent", "_left", "_right"

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right

    class Position(BinaryTree.Position):

        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        def __eq__(self, other: object) -> bool:
            return type(other) is type(self) and other._node is self._node

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError

        if p._container is not self:
            raise ValueError

        if p._node._parent is p._node:
            raise ValueError

        return p._node

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    def __init__(self):

        self._root = None
        self._size = 0

    def __len__(self):
        return self._size

    def root(self):
        return self._make_position(self._root)

    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):  # returns only children not descendants
        node = self._validate(p)

        count = 0
        if node._left is not None:
            count += 1

        if node._right is not None:
            count += 1

        return count

    def _add_root(self, e):

        if self._root is not None:
            raise ValueError("Root already there")

        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):

        node = self._validate(p)

        if node._left is not None:
            raise ValueError("Left present")

        self._size += 1
        node._left = self._Node(e)
        return self._make_position(node._left)

    def _add_right(self, p, e):

        node = self._validate(p)
        if node._right is not None:
            raise ValueError("Right Present")

        self._size += 1
        self._right = self._Node(e)
        return self._make_position(node._right)

    def _replace(self, p, e):

        node = self._validate(p)

        old = node._element
        node._element = e
        return old

    def _delete(self, p):

        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError("Two child can't delete")

        child = node._left if node._left else node._right

        if child is not None:

            child._parent = node._parent

        if node is self._root:
            self._root = child

        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element

    def _attach(self, p, t1, t2):

        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError("Not leaf can't attach")
        if not type(self) is type(t1) is type(t2):
            raise TypeError("Mismatch")

        self._size += len(t1) + len(t2)

        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0

        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0

In [1]:
class EulerTour:

    def __init__(self, tree):

        self._tree = tree

    def tree(self):

        return self._tree

    def execute(self):
        """Perform the tour and return any result from post visit of root."""
        if len(self.tree) > 0:
            return self._tour(self._tree.root(), 0, [])

    def _tour(self, p, d, path):

        self._hook_previst(p, d, path)
        results = []
        path.append(0)
        for c in self._tree.children(p):
            results.append(self._tour(c, d + 1, path))
            path[-1] += 1
        path.pop()
        answer = self._hook_postvisit(p, d, path, results)
        return answer

    def _hook_previsit(self, p, d, path):
        pass

    def _hook_postvisit(self, p, d, path, results):
        pass